In [1]:
"""Bond Pricing And Accrued Interest Calculation

Demonstrates pricing a bond and calculating it's accrued interest based on a user defined Bond Instrument.

Attributes
----------
instruments
aggregation
market data store
results store
quotes
"""


# Bond Pricing And Accrued Interest Calculation

This notebook will run through the following business use cases :
* [Pricing a bond using the built-in discounting LUSID model fed with a user supplied OIS yield curve.](#pricing_bond)
* [Calculating the accrued interest between coupon dates based on user defined bond instrument parameters such as the day count convention.](#accrued_interest)
* [Adding a transaction in a user defined bond instrument to a portfolio and subsequently valuing our portfolio.](#pricing_bond_portfolio)
* [Overriding the calculated accrued interest with user provided value and feeding it in to our bond valuation.](#accrual_override)
* [Valuing bond PV using an externally provided market quote for the bond.](#external_bond_price)

<br>

In doing so we'll cover the following LUSID concepts :
* [Defining a LUSID internal representation of a bond instrument based on user provided parameters.](#bond_definition)
* [Using the StructureMarketData store to hold your OIS yield curve data in way that enables it to be discovered during the bond valuation process.](#structured_market_data)
* [Configuring recipes to run built in LUSID bond valuation models that make use of the structured data (OIS Yield Curve) you provided.](#recipe_configuration)
* [Using aggregation requests to return the accrued interest as well as the PV of the bond based on our instrument definition.](#accrued_interest)
* [Using the StructuredResultData store to override the accrued interest calculation and instead use static values.](#structured_result_data)
* [Updating our recipes to make use of the StructuredResultData entries in your valuations.](#structured_result_data)
* [Upserting bond market price as quotes and configuring recipes that value bonds by using the market quotes.](#external_bond_price)

<br>

For this notebook example we'll work with an example Gilt 1.5% 47s:
* Coupon Rate : 1.5%
* Maturity Date: 22 Jul 2047
* Issue Date: 21 Sep 2016
* Coupon Dates : 22 Jan, 22 Jun
* Face Value : £1

## Setup LUSID and LUSID API objects.

In [2]:
import os
from datetime import datetime, timedelta

import lusid
from stop_execution import StopExecution
import pandas as pd
import pytz
from lusid import models
from lusid.utilities import ApiClientFactory
from lusidjam import RefreshingToken
from lusidtools.cocoon.cocoon_printer import (
    format_portfolios_response,
)


# Authenticate our user and create our API client
from lusidtools.cocoon import load_from_data_frame
secrets_path = os.getenv("FBN_SECRETS_PATH")

# Initiate an API Factory which is the client side object for interacting with LUSID APIs
api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename=secrets_path,
    app_name="LusidJupyterNotebook",
)

print ('LUSID Environment Initialised')
print ('LUSID SDK Version: ', api_factory.build(lusid.api.ApplicationMetadataApi).get_lusid_versions().build_version)

# Setup the apis we'll use in this notebook:
instruments_api = api_factory.build(lusid.api.InstrumentsApi)
complex_market_data_api = api_factory.build(lusid.api.ComplexMarketDataApi)
structured_result_data_api = api_factory.build(lusid.api.StructuredResultDataApi)

# Setup the scope we'll use in this notebook:
scope = "bond-pricing-nb"

/opt/conda/lib/python3.7/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


LUSID Environment Initialised
LUSID SDK Version:  0.0.1.0


In [3]:
# Settings and utility functions to display objects and responses more clearly.
pd.set_option('float_format', '{:,.3f}'.format)
def aggregation_result_to_dataframe(aggregation_results):
    return pd.DataFrame(aggregation_results, columns = ['Name', 'Effective At', 'Value'])

## Define our Gilt as a Bond Instrument in LUSID <a id="bond_definition"></a>

Let's start by defining our bond instrument in LUSID via the bond class in the models of the SDKs. Take a look
in the models package for other instruments currently supported (or see the [Bond Specification](https://www.lusid.com/api/swagger/index.html)).

We'll start by initialising the basic bond parameters for our Gilt:

In [4]:
coupon_rate = 0.015
start_date = datetime(2016, 9, 21, tzinfo=pytz.utc)
maturity_date = datetime(2047, 7, 22, tzinfo=pytz.utc)
dom_ccy = "GBP"
face_value = 1

trade_date = datetime(2020, 6, 22, tzinfo=pytz.utc)
effective_at = datetime(2020, 6, 23, tzinfo=pytz.utc)

Let's now move onto describing the conventions our bond instrument follows. Specifically we'll set up the behaviour of
our cash flows date schedule which includes setting the day count convention for calculating accrued interest and
handling cash flows landing on non business days. This behaviour is encapsulated in a FlowConventions object. For
details on supported tenors, day count and roll conventions see the [Bond Specification](https://www.lusid.com/api/swagger/index.html).

In [5]:
def create_bond_instrument_definition(start_date, maturity_date, dom_ccy, coupon_rate, face_value):
    instrument = models.Bond(
        start_date=start_date.isoformat(),
        maturity_date=maturity_date.isoformat(),
        dom_ccy=dom_ccy,
        coupon_rate=coupon_rate,
        principal=face_value,
        flow_conventions=models.FlowConventions(
            # coupon payment currency
            currency="GBP",
            # semi-annual coupon payments
            payment_frequency= "6M",
            # using an Actual/365 day count convention (other options : Act360, ActAct, ...
            day_count_convention="Act365",
            # modified following rolling convention (other options : ModifiedPrevious, NoAdjustment, EndOfMonth,...)
            roll_convention="ModifiedFollowing",
            # no holiday calendar supplied
            payment_calendars=[],
            reset_calendars=[],
            settle_days=2,
            reset_days=2,
        ),
        identifiers={},
        instrument_type="Bond"
    )
    return instrument

bond_instrument_definition = create_bond_instrument_definition(start_date, maturity_date, dom_ccy, coupon_rate, face_value)

Let's use our bond instrument definition to create an instance of our Gilt 1.5% 47 and upsert it as an instrument into LUSID:

In [6]:
def create_bond_instrument(instrument_id, instrument_name, bond_definition):
    bond_instrument_request = {instrument_id: models.InstrumentDefinition(
        # instrument display name
        name=instrument_name,
        # unique instrument identifier
        identifiers={"ClientInternal": models.InstrumentIdValue(instrument_id)},
        # our gilt instrument definition
        definition=bond_definition
    )}
    return instruments_api.upsert_instruments(bond_instrument_request)

instrument_creation_response = create_bond_instrument("gilt2047s", "gilt 1.5% 47s", bond_instrument_definition)
# retrieve the instrument id of our gilt to be used later when loading market quotes for the bond into LUSID.
gilt_2047_luid = instrument_creation_response.values['gilt2047s'].lusid_instrument_id


## Defining the Bond Valuation

Now that we have our bond instrument defined and upserted into LUSID we can move onto preparing to execute aggregations
in LUSID to value our bond.

Aggregations are configured in LUSID through the use of [Recipes]("https://support.finbourne.com/what-is-a-lusid-recipe-and-how-is-it-used").
Configuration describes functions such as how to source market data for specific asset classes, which pricing models to use, and where to locate static values that may
be used in the intermediate steps of the aggregation.

We'll begin with a recipe that simply informs the aggregation engine of which model to use to price our bond. The model
we'll use is LUSID's built-in "Discounting" model that prices our bond using an OIS yield curve. But before we can run our valuation
we need to cover how we supply LUSID with our OIS yield curve in a format it can understand.

### Complex Market Data <a id="complex_market_data"></a>

Complex Market Data expands on the simple uploading of market
quotes by allowing you to supply more complex market data into LUSID in a structured format.

In the case of our gilt we would like to price it using an OIS yield curve. The curve
consists of a set of discount factors across the maturities of the OIS term structure. While we've
specified the discount factors in this curve, lusid also supports building a curve from
a set of instruments and corresponding quotes.

Let's now load in the yield curves:

In [7]:
# scope used to store our market data
market_data_scope = "FinbourneMarketData"
# the market data supplier
market_supplier = 'Lusid'

def upsert_ois_yield_curve(scope, effective_at, market_asset):
    # provide the structured data file source and it's document format
    complex_market_data = models.DiscountFactorCurveData(
        base_date=datetime(2020, 6, 2, tzinfo=pytz.utc),
        dates = [datetime(2020, 6, 2, tzinfo=pytz.utc),datetime(2070, 6, 2, tzinfo=pytz.utc)],
        discount_factors= [1.0, 0.969944204112752],
        market_data_type="DiscountFactorCurveData"
    )

    # create a unique identifier for our OIS yield curves
    complex_id = models.ComplexMarketDataId(provider="Lusid",price_source=None,
                                            lineage="CurveVendorABC",
                                            effective_at=effective_at,
                                            market_asset=market_asset)

    upsert_request = models.UpsertComplexMarketDataRequest(market_data_id=complex_id,
                                                              market_data=complex_market_data)

    # https://www.lusid.com/docs/api#operation/UpsertComplexMarketData
    response = complex_market_data_api.upsert_complex_market_data(
        scope=scope,
        request_body={market_asset : upsert_request}
    )

    if response.failed:
        raise StopExecution("Failed to upload yield curve {response.failed}")

    print(f"{market_asset} yield curve uploaded into scope={scope}")

upsert_ois_yield_curve(market_data_scope, effective_at, "GBP/GBPOIS")
upsert_ois_yield_curve(market_data_scope, effective_at, "GBP/6M")

GBP/GBPOIS yield curve uploaded into scope=FinbourneMarketData
GBP/6M yield curve uploaded into scope=FinbourneMarketData


### What Bond Pricing Model To Use?
To value our Gilt we need to tell LUSID how to answer the following questions - What bond pricing model to use? And where
should LUSID source the market data to required to properly execute the model? To do so we define a Recipe that has two
main constituents, the PricingContext and the MarketContext.

We start with the PricingContext which is used to select the pricing model and add any additional parameters that configure the
model behaviour. See the [Swagger spec]("https://www.lusid.com/api/swagger/index.html") under "PricingContext" for a detailed
description of the parameters.

In [8]:
def create_discounting_bond_pricing_context():
    return models.PricingContext(
        # select the "Discounting" model for bond pricing
        model_rules=[
            models.VendorModelRule(
                supplier="Lusid",
                model_name="Discounting",
                instrument_type="Bond",
                parameters="{}"
            )
        ]
    )

pricing_context = create_discounting_bond_pricing_context()

### Where should Market Data be sourced?

The MarketContext is how we inform LUSID where to retrieve market data for a given aggregation. In our case we need to
tell LUSID where our OIS yield curve was stored. See the [Swagger spec]("https://www.lusid.com/api/swagger/index.html#model-MarketContext")
under "MarketContext" for a detailed description of the parameters.

In [9]:
def create_market_context():
    return models.MarketContext(
        # set rules for where we should resolve our rates data. In our case the OIS yield curves.
        market_rules=[
            models.MarketDataKeyRule(
                key="Rates.*.*",
                data_scope=market_data_scope,
                supplier=market_supplier,
                quote_type='Rate',
                field='Mid',
                quote_interval='2D')
        ],
        # control default options for resolving market data. In our case simply default to the LUSID market_supplier
        # and market data scope we defined earlier.
        options=models.MarketOptions(
            default_supplier=market_supplier,
            default_scope=market_data_scope)
    )

    return market_context

market_context = create_market_context()

### Configure our Bond Pricing Recipe <a id="recipe_configuration"></a>

With our PricingContext and MarketContext defined we're now ready to configure our bond valuation recipe:

In [10]:
def create_discount_bond_pricing_recipe(scope, market_context, pricing_context):

    return models.ConfigurationRecipe(
        scope=scope,
        code="discounting-bond",
        description="Price bond using discounting model",
        market=market_context,
        pricing=pricing_context
    )

discount_bond_pricing_config_recipe = create_discount_bond_pricing_recipe(scope, market_context, pricing_context)

# Upsert recipe to LUSID
upsert_recipe_request = models.UpsertRecipeRequest(configuration_recipe=discount_bond_pricing_config_recipe)
response = api_factory.build(lusid.api.ConfigurationRecipeApi).upsert_configuration_recipe(upsert_recipe_request)


## Pricing our Bond <a id="pricing_bond"></a>

Let's quickly summarise our current state:
 * We've defined a Gilt 1.5% 47s bond (including defining it's date conventions).
 * We've loaded the OIS yield curve into the Structure Market Data store.
 * We've setup our Recipe configuring of how we would like to price our bond and where to source our required market data.

At this point we hold no position in the bond in our portfolio but would simply like to value it using our internal bond
pricing model as defined in the recipe. LUSID offers the capability to run aggregations on non-existing positions through the
use of inline portfolios. Inline portfolios are defined as a set of weighted instruments which can be used for example to define
an Index. However in our simplified example we only have the one constituent which is our Gilt.

### Run an Aggregation to Price our Bond

In [11]:
def run_bond_pricing_aggregation(bond_instrument_definition, discount_bond_pricing_config_recipe, effective_at):
    # setup weighted instrument (only our gilt definition)
    weighted_instrument_gilt = models.WeightedInstrument(quantity=1, instrument=bond_instrument_definition, holding_identifier="myholding_gilt")
    
    # As we're running an inline valuation we must pass in our weighted instruments
    inline_valuation_request = models.InlineValuationRequest(
        recipe_id=models.ResourceId(
            scope=discount_bond_pricing_config_recipe.scope,
            code=discount_bond_pricing_config_recipe.code
        ),
        metrics=[
            models.AggregateSpec(key='Valuation/PV', op='Value'),
        ],
        valuation_schedule=models.ValuationSchedule(
            effective_at=effective_at.isoformat()
        ),
        instruments=[weighted_instrument_gilt]
    )

    # https://www.lusid.com/docs/api#operation/GetValuationOfWeightedInstruments
    return api_factory.build(lusid.api.AggregationApi).get_valuation_of_weighted_instruments(
        inline_valuation_request=inline_valuation_request)



result = run_bond_pricing_aggregation(bond_instrument_definition, discount_bond_pricing_config_recipe, effective_at)
bond_pv = result.data[0]['Valuation/PV']
aggregation_result_to_dataframe([
    ['Bond PV', effective_at, bond_pv]
])

,Name,Effective At,Value
0,Bond PV,2020-06-23 00:00:00+00:00,1.399


### Pricing our Bond the Following Day

We've now priced our Gilt using the LUSID internal bond "Discounting" model. As our recipe is already setup
we can seamlessly revalue our bond for the following day:

In [12]:
effective_at_t_plus_one = effective_at + timedelta(days=1)
aggregation_result = run_bond_pricing_aggregation(bond_instrument_definition, discount_bond_pricing_config_recipe, effective_at_t_plus_one)
bond_pv_at_t_plus_one = aggregation_result.data[0]['Valuation/PV']

aggregation_result_to_dataframe([
    ['Bond PV', effective_at, bond_pv],
    ['Bond PV', effective_at_t_plus_one, bond_pv_at_t_plus_one]
])

,Name,Effective At,Value
0,Bond PV,2020-06-23 00:00:00+00:00,1.399
1,Bond PV,2020-06-24 00:00:00+00:00,1.399


## Accrued Interest <a id="accrued_interest"></a>

The above example shows our bond price changing from one day to the next using the Discounting model. This can be explained by the accrued
interest on the Gilt. Recall that when we defined our gilt we set FlowConventions that contain all the information needed to calculate
accrued interest between coupon dates.


### Returning the Calculated Accrued Interest
Note that the The 'Valuation/PV' aggregation firstly generates a price using the "Discounting" model before
adding the calculated accrued interest and returning the PV. To get back the accrued interest that was calculated we simply need to update
our aggregation request to include 'Holding/default/Accrual':


In [13]:
def run_bond_pricing_aggregation_and_accrued(bond_instrument_definition, discount_bond_pricing_config_recipe, effective_at):
    weighted_instrument_gilt = models.WeightedInstrument(quantity=1, instrument=bond_instrument_definition, holding_identifier="myholding_gilt")

        # As we're running an inline valuation we must pass in our weighted instruments
    inline_valuation_request = models.InlineValuationRequest(
        recipe_id=models.ResourceId(
            scope=discount_bond_pricing_config_recipe.scope,
            code=discount_bond_pricing_config_recipe.code
        ),
        metrics=[
            models.AggregateSpec(key='Valuation/PV', op='Value'),
            # Ensure the calculated accrual is returned
            models.AggregateSpec(key='Valuation/Accrued', op='Value')
        ],
        valuation_schedule=models.ValuationSchedule(
            effective_at=effective_at.isoformat()
        ),
        instruments=[weighted_instrument_gilt]
    )

    # https://www.lusid.com/docs/api#operation/GetValuationOfWeightedInstruments
    return api_factory.build(lusid.api.AggregationApi).get_valuation_of_weighted_instruments(
        inline_valuation_request=inline_valuation_request)


# bond pv and accrued interest at effective date
aggregation_result = run_bond_pricing_aggregation_and_accrued(bond_instrument_definition, discount_bond_pricing_config_recipe, effective_at)
bond_pv = aggregation_result.data[0]['Valuation/PV']
accrued_interest = aggregation_result.data[0]['Valuation/Accrued']

# bond pv and accrued interest day after
aggregation_result = run_bond_pricing_aggregation_and_accrued(bond_instrument_definition, discount_bond_pricing_config_recipe, effective_at_t_plus_one)
bond_pv_t_plus_one = aggregation_result.data[0]['Valuation/PV']
accrued_interest_t_plus_one = aggregation_result.data[0]['Valuation/Accrued']

aggregation_result_to_dataframe([
    ['Bond PV', effective_at, bond_pv],
    ['Bond PV', effective_at_t_plus_one, bond_pv_t_plus_one],
    ['Bond PV Dtd', effective_at_t_plus_one, bond_pv_t_plus_one-bond_pv],
    ['Accrued Interest', effective_at, accrued_interest],
    ['Accrued Interest', effective_at_t_plus_one, accrued_interest_t_plus_one],
    ['Accrued Interest Dtd', effective_at_t_plus_one, accrued_interest_t_plus_one-accrued_interest]
])

,Name,Effective At,Value
0,Bond PV,2020-06-23 00:00:00+00:00,1.399
1,Bond PV,2020-06-24 00:00:00+00:00,1.399
2,Bond PV Dtd,2020-06-24 00:00:00+00:00,0.000
3,Accrued Interest,2020-06-23 00:00:00+00:00,0.006
4,Accrued Interest,2020-06-24 00:00:00+00:00,0.006
5,Accrued Interest Dtd,2020-06-24 00:00:00+00:00,0.000


## Adding a Bond Position to our Portfolio

Now that we've covered how to define and price a bond instrument let's move onto adding a bond position to our portfolio.
We'll then use the same recipe we defined to price the bond but this time use it to value our entire bond position within
our portfolio. So we're using the same Recipe (i.e same Bond pricing model and data sources) but different aggregation parameters

Brief summary of what we'll aim to do:
* Create a portfolio to hold our Bond position.
* Create a buy transaction on our holding. We'll also make use of the accrual calculation we just covered to help us
setup a realistic transaction price and consideration for our test case.
* Run an aggregation to calculate the value of our portfolio with the accrued interest. We'll also review how this aggregation
on our portfolio differs to the the previous inline aggregation.

### Setting up our Portfolio

In [14]:
portfolio = "simple-bond-portfolio-01"

def create_portfolio(scope, portfolio_code, portfolio_name, portfolio_ccy):
    pfs = [[portfolio_code, portfolio_name, portfolio_ccy]]
    pf_df = pd.DataFrame(pfs, columns=['portfolio_code', 'portfolio_name', 'base_currency'])

    portfolio_mapping = {
        "required": {
            "code": "portfolio_code",
            "display_name": "portfolio_name",
            "base_currency": "base_currency",
        },
        "optional": {"created": "$2020-01-01T00:00:00+00:00"},
    }
    result = load_from_data_frame(
        api_factory=api_factory,
        scope=scope,
        data_frame=pf_df,
        mapping_required=portfolio_mapping["required"],
        mapping_optional=portfolio_mapping["optional"],
        file_type="portfolios",
        sub_holding_keys=[],
    )
    succ, failed = format_portfolios_response(result)

    if not failed.empty:
        raise StopExecution(failed)

    return succ

create_portfolio(scope, portfolio, portfolio, "GBP")

,successful items
0,simple-bond-portfolio-01


### Creating our Transaction

We now need to create and upsert our buy transaction. Earlier we priced the Present Value of our bond (approx 138.66) which
included accrued interest of around 0.00062. For our example transaction let's set a market price of 137.00 and we're looking
for a position size with a notional of 75,000,000. We'll use the accrued interest calculated by LUSID to come up with a dirty price
and consideration for this particular example:

In [15]:
# retrieve accrued interest
aggregation_result = run_bond_pricing_aggregation_and_accrued(bond_instrument_definition, discount_bond_pricing_config_recipe, effective_at)
bond_unit_pv = aggregation_result.data[0]['Valuation/PV']
accrued_unit_interest = aggregation_result.data[0]['Valuation/Accrued']

# setup our transaction
notional = 75000000
clean_price = 1.37
dirty_price = clean_price + accrued_unit_interest
consideration = notional * dirty_price

def upsert_buy_transaction(scope, txn_id, instrument_id, trade_date, effective_at, portfolio, clean_price):
    gilt_transaction_request = models.TransactionRequest(
        transaction_id=txn_id,
        type="Buy",
        instrument_identifiers={"Instrument/default/ClientInternal": instrument_id},
        transaction_date=trade_date.isoformat(),
        settlement_date=effective_at.isoformat(),
        units=notional,
        transaction_price=models.TransactionPrice(price=clean_price, type="Price"),
        total_consideration=models.CurrencyAndAmount(amount=consideration, currency="GBP"),
        exchange_rate=1,
        transaction_currency="GBP"
    )

    response = api_factory.build(lusid.api.TransactionPortfoliosApi).upsert_transactions(scope=scope,
                                                                                         code=portfolio,
                                                                                         transaction_request=[gilt_transaction_request])

upsert_buy_transaction(scope, "GiltTXN001", "gilt2047s", trade_date, effective_at, portfolio, clean_price)


## Valuing our Portfolio <a id="pricing_bond_portfolio"></a>

We're now ready to run our aggregation and value our bond portfolio. It's important to note that this is no longer an
inline aggregation. This means the aggregation we're now running isn't only against the bond instrument definition, but instead
is being run against a portfolio within a specific scope. For this reason we no longer require the use of a weighted instrument
or the need to generate a specific inline aggregation request. Our request is simpler and only requires our recipe, portfolio and scope.

### Run an Aggregation to Price our Portfolio With Bond Holdings

In [16]:
def run_bond_pricing_aggregation_on_portfolio(scope, portfolio, recipe, effective_at):
    valuation_request = models.ValuationRequest(
        recipe_id=models.ResourceId(
            scope=recipe.scope,
            code=recipe.code
        ),
          metrics=[
            models.AggregateSpec(key='Valuation/PV', op='Value'),
            models.AggregateSpec(key='Valuation/Accrued', op='Value'),
            models.AggregateSpec(key='Valuation/InstrumentAccrued', op='Value')

        ],
        valuation_schedule=models.ValuationSchedule(effective_at=effective_at.isoformat()),
        portfolio_entity_ids=[
            models.PortfolioEntityId(
                scope=scope,
                code=portfolio
            )
        ]
    )

    return api_factory.build(lusid.api.AggregationApi).get_valuation(valuation_request=valuation_request)



results = run_bond_pricing_aggregation_on_portfolio(scope, portfolio, discount_bond_pricing_config_recipe, effective_at)
bond_pv = results.data[0]['Valuation/PV']
accrued_interest = results.data[0]['Valuation/Accrued']

aggregation_result_to_dataframe([
    ['Portfolio PV inc Accrd Int', effective_at, bond_pv],
    ['Portfolio Accrued Interest', effective_at, accrued_interest]
])

,Name,Effective At,Value
0,Portfolio PV inc Accrd Int,2020-06-23 00:00:00+00:00,"104,926,020.472"
1,Portfolio Accrued Interest,2020-06-23 00:00:00+00:00,"472,870.879"


### Pricing our Bond the Following Day

As in our previous example let's move forward a day to view the change in our Portfolio PV and Accrued Interest:

In [17]:
effective_at_plus_one = effective_at + timedelta(days=1)
results = run_bond_pricing_aggregation_on_portfolio(scope, portfolio, discount_bond_pricing_config_recipe, effective_at_plus_one)
bond_pv_t_plus_one = results.data[0]['Valuation/PV']
accrued_interest_t_plus_one = results.data[0]['Valuation/Accrued']

aggregation_result_to_dataframe([
    ['Portfolio PV inc Accrd Int', effective_at, bond_pv],
    ['Portfolio PV inc Accrd Int', effective_at_t_plus_one, bond_pv_t_plus_one],
    ['Portfolio PV inc Accrd Int Dtd', effective_at_t_plus_one, bond_pv_t_plus_one - bond_pv],
    ['Portfolio Accrued Interest', effective_at, accrued_interest],
    ['Portfolio Accrued Interest', effective_at_t_plus_one, accrued_interest_t_plus_one],
    ['Portfolio Accrued Interest Dtd', effective_at_t_plus_one, accrued_interest_t_plus_one - accrued_interest]
])

,Name,Effective At,Value
0,Portfolio PV inc Accrd Int,2020-06-23 00:00:00+00:00,"104,926,020.472"
1,Portfolio PV inc Accrd Int,2020-06-24 00:00:00+00:00,"104,929,285.678"
2,Portfolio PV inc Accrd Int Dtd,2020-06-24 00:00:00+00:00,"3,265.206"
3,Portfolio Accrued Interest,2020-06-23 00:00:00+00:00,"472,870.879"
4,Portfolio Accrued Interest,2020-06-24 00:00:00+00:00,"475,961.538"
5,Portfolio Accrued Interest Dtd,2020-06-24 00:00:00+00:00,"3,090.659"


## Accrual Overrides <a id="accrual_override"></a>

So far we've relied on LUSID to run it's internal bond pricing model as well as calculate accruals based on the
conventions we defined in the Gilt instrument definition. However you may need to use a different value for accrued interest - one
that has been calculated externally for example. To do so we need to address two concerns. How do we load a one off accrual
into LUSID? And how do we ensure LUSID uses that accrual during aggregation as oppose to reverting to calculating it
on the fly as in the previous examples.

### Structured Result Data <a id="structured_result_data"></a>

The [Structured Result Store]("https://support.finbourne.com/how-do-i-store-and-retrieve-structured-market-data-documents") is
a location to store non quote data that may nevertheless be used in an aggregation. Examples include YTD performance on
an Index, sensitivities of a Swap, or in our case the accrued interest on a bond between coupon dates.

Just as with we did with our OIS yield curves in the Structured Market Data store we need to ensure that the Results we
upsert can be resolved by LUSID during an aggregation. To do so all entries into the Result store must be defined with a
corresponding key that uniquely identifies what the value relates to.

In [18]:
result_data_scope = "Finbourne-Examples"
accrual_result_id = "GiltAccrual"

def upsert_structured_result_data_overrides(effective_at, accrual_result_id, instrument_id):
    # mock an entry from  a csv file
    accrual_document = "LusidInstrumentId,Accrual,AccrualCcy" + "\r\n" + f"{instrument_id},0.069109979,GBP"

    data_map_key = models.DataMapKey(
        code = "sample-data-map",
        version = "1.0.1"
    )
    
    try:
        structured_result_data_api.create_data_map(
            scope = result_data_scope,
            request_body = {
                "data-map": models.CreateDataMapRequest(
                    id=data_map_key,
                    data=models.DataMapping(
                        data_definitions=[
                            models.DataDefinition(address="UnitResult/LusidInstrumentId", name="LusidInstrumentId", data_type="String", key_type="Unique"),
                            models.DataDefinition(address="UnitResult/Valuation/InstrumentAccrued", data_type="Result0D", key_type="CompositeLeaf"),
                            models.DataDefinition(address="UnitResult/Valuation/InstrumentAccrued/Amount", name="Accrual", data_type="Decimal", key_type="Leaf"),
                            models.DataDefinition(address="UnitResult/Valuation/InstrumentAccrued/Ccy", name="AccrualCcy", data_type="String", key_type="Leaf"),
                        ]
                    )
                )
            }
        )
    except:
        print("DataMaps are immutable - a datamap under this key already exists")
    
    # create the result data object from our loaded csv file and definition of the format
    accrual_result = models.StructuredResultData(
        document_format="CSV",
        version="1.0.0",
        name="IRS accrual",
        document=accrual_document,
        data_map_key=data_map_key
    )
    
    # create a unique identifier for our accrual to ensure it can be properly resolved during aggregation
    accrual_result_id = models.StructuredResultDataId(
        source="Client",
        code=accrual_result_id,
        effective_at=effective_at.isoformat(),
        result_type="UnitResult/Analytic"
    )

    # create structured request
    structured_request = models.UpsertStructuredResultDataRequest(
        id=accrual_result_id,
        data=accrual_result
    )

    # https://www.lusid.com/docs/api#operation/UpsertStructuredResultData
    response = structured_result_data_api.upsert_structured_result_data(
        scope=result_data_scope,
        request_body={"AccrualOR1": structured_request}
    )
    if response.failed:
        raise StopExecution(f"Failed to upsert result data: {response.failed}")

    print(f"Upserted accrual result {accrual_result_id.code}.")

upsert_structured_result_data_overrides(effective_at, accrual_result_id, gilt_2047_luid)

Upserted accrual result GiltAccrual.


### Setting Up Our Result Data Rule

Now that the accrual override is in LUSID we need to address our second concern of notifying the aggregation to override the
accrual calculation with our value when required. We can achieve this via the use of a "Result Data Rule". These rules
rely on a pattern that when matched signals to the aggregation process that the stored result data should be used over any
calculated value.

In [19]:
def create_accrual_result_data_rule(result_data_scope, accrual_result_id):
    accrual_key_rule = models.ResultDataKeyRule(
        # identifies which patterns of results this rule should be applied for.
        resource_key="UnitResult/Valuation/InstrumentAccrued",
        supplier="Client",
        data_scope=result_data_scope,
        document_code=accrual_result_id,
        document_result_type="UnitResult/Analytic",
        result_key_rule_type="ResultDataKeyRule"
    )

    return accrual_key_rule


accrual_key_rule = create_accrual_result_data_rule(result_data_scope, accrual_result_id)

### Applying our Result Data Rule to a Recipe via PricingContext

With the rule created we now need to instruct LUSID to apply this rule. To do so we simply add our rule to a an updated PricingContext definition
(which you recall tells LUSID what models to use for the aggregation).

In [20]:
def create_discounting_bond_pricing_context_with_accrual_override(accrual_key_rule):
    return models.PricingContext(
        model_rules=[
            models.VendorModelRule(
                supplier="Lusid",
                model_name="Discounting",
                instrument_type="Bond",
                parameters="{}"
            )
        ],
        result_data_rules=[accrual_key_rule]
    )

accrual_override_pricing_context = create_discounting_bond_pricing_context_with_accrual_override(accrual_key_rule)

### Creating a new Recipe with our Accrual Override Rule

As we've updated the pricing context we need to generate a new recipe for our portfolio valuation :

In [21]:

def create_bond_pricing_recipe_with_accrual_override(scope, market_context, pricing_context):

    return models.ConfigurationRecipe(
        scope=scope,
        code="discounting-bond-acc-override",
        description="Price bond using discounting model but override accruals",
        market=market_context,
        pricing=pricing_context
    )

accrual_override_config_recipe = create_bond_pricing_recipe_with_accrual_override(scope, market_context, accrual_override_pricing_context)

# Upsert recipe to LUSID
upsert_recipe_request = models.UpsertRecipeRequest(configuration_recipe=accrual_override_config_recipe)
response = api_factory.build(lusid.api.ConfigurationRecipeApi).upsert_configuration_recipe(upsert_recipe_request)

### Price our Portfolio using the Accrual Override

Let's rerun the portfolio aggregation but this time with our accrual override and compare the results to running without an override

In [22]:
# run the aggregation with an accrual override (using accrual override recipe)
results_with_override = run_bond_pricing_aggregation_on_portfolio(scope, portfolio, accrual_override_config_recipe, effective_at)
portfolio_pv_with_override = results_with_override.data[0]['Valuation/PV']
accrued_interest_with_override = results_with_override.data[0]['Valuation/Accrued']

# and now run without an override (using previously configured recipe).
results_no_override = run_bond_pricing_aggregation_on_portfolio(scope, portfolio, discount_bond_pricing_config_recipe, effective_at)
portfolio_pv_no_override = results_no_override.data[0]['Valuation/PV']
accrued_interest_no_override = results_no_override.data[0]['Valuation/Accrued']

aggregation_result_to_dataframe([
    ['Portfolio PV inc Accrd Int (With Override)', effective_at, portfolio_pv_with_override],
    ['Portfolio PV inc Accrd Int (No Override)', effective_at, portfolio_pv_no_override],
    ['Portfolio Accrued Interest  (With Override)', effective_at, accrued_interest_with_override],
    ['Portfolio Accrued Interest (No Override)', effective_at, accrued_interest_no_override],
])

,Name,Effective At,Value
0,Portfolio PV inc Accrd Int (With Override),2020-06-23 00:00:00+00:00,"109,636,398.018"
1,Portfolio PV inc Accrd Int (No Override),2020-06-23 00:00:00+00:00,"104,926,020.472"
2,Portfolio Accrued Interest (With Override),2020-06-23 00:00:00+00:00,"5,183,248.425"
3,Portfolio Accrued Interest (No Override),2020-06-23 00:00:00+00:00,"472,870.879"


## Bond Price Override <a id="external_bond_price"></a>

Up until now we have priced our bond portfolio using LUSID's internal "Discounting" model. We'll now take a different approach and
upsert the market price of the Gilt into LUSID as a quote. Continuing with our previous example where we processed a transaction at a clean price of 137.00,
now assume the market price of the bond has dropped to 135.00


### Load our Gilt Market Price into LUSID
Let's firstly upsert the latest market price as a quote into LUSID ensuring we map it to our specific Gilt via the instrument identifier:

In [23]:
def upsert_external_bond_price_as_quote(bond_price):
    spot_quote = models.UpsertQuoteRequest(
        quote_id=models.QuoteId(
            quote_series_id=models.QuoteSeriesId(
                provider=market_supplier,
                instrument_id=gilt_2047_luid,
                instrument_id_type='LusidInstrumentId',
                quote_type='Price',
                field='Mid'),
            effective_at=effective_at,
        ),
        metric_value=models.MetricValue(
            value=bond_price,
            unit='GBP'),
        lineage='InternalSystem')

    response = api_factory.build(lusid.api.QuotesApi).upsert_quotes(
        scope=market_data_scope,
        request_body={"1": spot_quote})

    if response.failed:
        raise StopExecution(f"Failed to upload yield curve:{response.failed}")

    print(f"Gilt 20147 @{bond_price} uploaded into Quote store.")

upsert_external_bond_price_as_quote(1.35)

Gilt 20147 @1.35 uploaded into Quote store.


### Add a Market Rule to help LUSID resolve our Gilt Price

As we now want to run a portfolio valuation using a different pricing model for bonds we need a new recipe. Recall the two key components of the recipe
are the MaketContext and PricingContext. As we now have a new source of Market data, the quote we've inserted, we need to inform LUSID of where to locate it
during an aggregation. So in addition to the Rate rule we had in our precious MarketContext definition we add a new market rule for "Price":

In [24]:
def create_static_bond_pricing_market_context():
    return models.MarketContext(
        market_rules=[
            # additional rule to resolve our quote
            models.MarketDataKeyRule(
                key='Quote.LusidInstrumentId.*',
                supplier=market_supplier,
                data_scope=market_data_scope,
                quote_type='Price',
                field='Mid'),
            models.MarketDataKeyRule(
                key="Rates.*.*",
                data_scope=market_data_scope,
                supplier=market_supplier,
                quote_type='Rate',
                field='Mid')
        ],
        options=models.MarketOptions(
            default_supplier=market_supplier,
            default_scope=market_data_scope)
    )

    return market_context

static_bond_pricing_market_context = create_static_bond_pricing_market_context()

### Update the Bond Pricing Model used in the Aggregation

We need to instruct LUSID that we would like to used the quoted price of the bond and not use
the "Discounting" model. So we create a new PricingContext that uses a "SimpleStatic" model. To calculate PV of our bond
this model simply retrieves the market price and then adds the accrued interest (which itself could either be calculated or loaded
from the Structured Result Store as we covered earlier).

In [25]:
def create_static_bond_pricing_context():
    return models.PricingContext(
        # the default behaviour does not allow looking up data for pricing instruments so we must allow it.
        options=models.PricingOptions(
            allow_any_instruments_with_sec_uid_to_price_off_lookup=True
        ),
        model_rules=[
            models.VendorModelRule(
                supplier="Lusid",
                model_name="SimpleStatic",
                instrument_type="Bond",
                parameters="{}"
            )
        ]
    )

static_bond_pricing_context = create_static_bond_pricing_context()

### Create a new Recipe with our new Market Rules and Bond Pricing Model

With our updated Market and Pricing Contexts we can now generate a new recipe:

In [26]:
def create_static_bond_pricing_recipe(scope, market_context, pricing_context):

    return models.ConfigurationRecipe(
        scope=scope,
        code="static-bond",
        description="Price bond using prices from the quote store.",
        market=market_context,
        pricing=pricing_context
    )

static_bond_price_config_recipe = create_static_bond_pricing_recipe(scope, static_bond_pricing_market_context, static_bond_pricing_context)

# Upsert recipe to LUSID
upsert_recipe_request = models.UpsertRecipeRequest(configuration_recipe=static_bond_price_config_recipe)
response = api_factory.build(lusid.api.ConfigurationRecipeApi).upsert_configuration_recipe(upsert_recipe_request)

### Run Aggregations to Price our Portfolio using Static and Discounting Bond Pricing Models

Let's rerun our portfolio valuation using our static bond price model and compare the results to the valuations using our discounting model:

In [27]:
results_using_static_price = run_bond_pricing_aggregation_on_portfolio(scope, portfolio, static_bond_price_config_recipe, effective_at)
portfolio_pv_with_static_price = results_using_static_price.data[0]['Valuation/PV']
accrued_interest_with_static_price = results_using_static_price.data[0]['Valuation/Accrued']

results_using_discounting = run_bond_pricing_aggregation_on_portfolio(scope, portfolio, discount_bond_pricing_config_recipe, effective_at)
portfolio_pv_with_discounting = results_using_discounting.data[0]['Valuation/PV']
accrued_interest_with_discounting = results_using_discounting.data[0]['Valuation/Accrued']

aggregation_result_to_dataframe([
    ['Portfolio PV inc Accrd Int (Static Model)', effective_at, portfolio_pv_with_static_price],
    ['Portfolio PV inc Accrd Int (Discounting Model)', effective_at, portfolio_pv_with_discounting],
    ['Portfolio Accrued Interest  (Static Model)', effective_at, accrued_interest_with_static_price],
    ['Portfolio Accrued Interest (Discounting Mode)', effective_at, accrued_interest_with_discounting],
])

,Name,Effective At,Value
0,Portfolio PV inc Accrd Int (Static Model),2020-06-23 00:00:00+00:00,"101,722,870.879"
1,Portfolio PV inc Accrd Int (Discounting Model),2020-06-23 00:00:00+00:00,"104,926,020.472"
2,Portfolio Accrued Interest (Static Model),2020-06-23 00:00:00+00:00,"472,870.879"
3,Portfolio Accrued Interest (Discounting Mode),2020-06-23 00:00:00+00:00,"472,870.879"


### Revalue Portfolio with an Increased Gilt Price

Update the market price of the bond to 139.00

In [28]:
upsert_external_bond_price_as_quote(1.39)

results_using_static_price = run_bond_pricing_aggregation_on_portfolio(scope, portfolio, static_bond_price_config_recipe, effective_at)
portfolio_pv_with_static_price = results_using_static_price.data[0]['Valuation/PV']
accrued_interest_with_static_price = results_using_static_price.data[0]['Valuation/Accrued']

aggregation_result_to_dataframe([
    ['Portfolio PV inc Accrd Int (Static Model)', effective_at, portfolio_pv_with_static_price],
    ['Portfolio PV inc Accrd Int (Discounting Model)', effective_at, portfolio_pv_with_discounting],
    ['Portfolio Accrued Interest  (Static Model)', effective_at, accrued_interest_with_static_price],
    ['Portfolio Accrued Interest (Discounting Mode)', effective_at, accrued_interest_with_discounting],
])

Gilt 20147 @1.39 uploaded into Quote store.


,Name,Effective At,Value
0,Portfolio PV inc Accrd Int (Static Model),2020-06-23 00:00:00+00:00,"104,722,870.879"
1,Portfolio PV inc Accrd Int (Discounting Model),2020-06-23 00:00:00+00:00,"104,926,020.472"
2,Portfolio Accrued Interest (Static Model),2020-06-23 00:00:00+00:00,"472,870.879"
3,Portfolio Accrued Interest (Discounting Mode),2020-06-23 00:00:00+00:00,"472,870.879"


### Revalue Portfolio with the same Clean Gilt Price

Finally let's update the market price to equal the clean price as per the "Discounting" model. In this case we arrive at
the same portfolio PV using two different bond pricing models

In [29]:
# retrieve unit pv and accrued interest
aggregation_result = run_bond_pricing_aggregation_and_accrued(bond_instrument_definition, discount_bond_pricing_config_recipe, effective_at)
clean_as_per_disc_model = aggregation_result.data[0]['Valuation/PV'] - aggregation_result.data[0]['Valuation/Accrued']
upsert_external_bond_price_as_quote(clean_as_per_disc_model)

results_using_static_price = run_bond_pricing_aggregation_on_portfolio(scope, portfolio, static_bond_price_config_recipe, effective_at)
portfolio_pv_with_static_price = results_using_static_price.data[0]['Valuation/PV']
accrued_interest_with_static_price = results_using_static_price.data[0]['Valuation/Accrued']


aggregation_result_to_dataframe([
    ['Portfolio PV inc Accrd Int (Static Model)', effective_at, portfolio_pv_with_static_price],
    ['Portfolio PV inc Accrd Int (Discounting Model)', effective_at, portfolio_pv_with_discounting],
    ['Portfolio Accrued Interest  (Static Model)', effective_at, accrued_interest_with_static_price],
    ['Portfolio Accrued Interest (Discounting Mode)', effective_at, accrued_interest_with_discounting],
])

Gilt 20147 @1.3927086612381778 uploaded into Quote store.


,Name,Effective At,Value
0,Portfolio PV inc Accrd Int (Static Model),2020-06-23 00:00:00+00:00,"104,926,020.472"
1,Portfolio PV inc Accrd Int (Discounting Model),2020-06-23 00:00:00+00:00,"104,926,020.472"
2,Portfolio Accrued Interest (Static Model),2020-06-23 00:00:00+00:00,"472,870.879"
3,Portfolio Accrued Interest (Discounting Mode),2020-06-23 00:00:00+00:00,"472,870.879"
